In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install torch
!pip install datasets
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [ ]:
from transformers import EncoderDecoderModel
from transformers import AutoTokenizer
import csv
import torch
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder_tokenizer = AutoTokenizer.from_pretrained("SIKU-BERT/sikuroberta")
decoder_tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')

test = EncoderDecoderModel.from_pretrained('model').to(device)

def generate_summary(data):
    inputs = encoder_tokenizer(data["data"], padding="max_length", truncation=True, max_length=192, return_tensors="pt")

    input_ids=inputs.input_ids.to(device)

    attention_mask = inputs.attention_mask.to(device)

    outputs = test.generate(input_ids,attention_mask=attention_mask,max_length=192)

    data["output"] = decoder_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return data


headers = ['id', 'source', 'translation']

output_data=[]
tmp=[]
with open("/content/drive/MyDrive/Colab Notebooks/test_utf8.tsv","r",encoding='utf-8') as tsv:
    reader = csv.reader(tsv, delimiter='\t')
    for row in reader:
        if row[0]=="id":
            continue
        tmp.append(row[1])

data=Dataset.from_dict({"data": tmp})

data = data.map(
    generate_summary,
    batched=True,
    batch_size=20,
    remove_columns=["data"]
)

temp=[]
count=1
for x,y in zip(data["output"],tmp):
  temp.append([count,y,x])
  count+=1

with open("validate.tsv","w",encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='\t', lineterminator='\n')
    writer.writerow(headers)
    writer.writerows(temp)